In [4]:
import numpy as np
import pandas as pd
#flist = ['05_24', '05_25', '05_26', '05_27', '05_28', ]
flist = ['07_23']
data = pd.DataFrame()
first = True
for file in flist:
    if first:
        data = pd.read_pickle('/data/model_file/data_'+file+'.pkl')
        first = False
        with open('/data/model_file/y_'+file+'.npy', 'rb') as f:
            y = np.load(f)
        with open('/data/model_file/y3_'+file+'.npy', 'rb') as f:
            y3 = np.load(f)
        with open('/data/model_file/y4_'+file+'.npy', 'rb') as f:
            y4 = np.load(f)
    else:
        tmp = pd.read_pickle('/data/model_file/data_'+file+'.pkl')
        data = pd.concat([data, tmp])
        with open('/data/model_file/y_'+file+'.npy', 'rb') as f:
            tmp = np.load(f)
        y = np.concatenate((y, tmp), axis=0)
        with open('/data/model_file/y3_'+file+'.npy', 'rb') as f:
            tmp = np.load(f)
        y3 = np.concatenate((y3, tmp), axis=0)
        with open('/data/model_file/y4_'+file+'.npy', 'rb') as f:
            tmp = np.load(f)
        y4 = np.concatenate((y4, tmp), axis=0)
    

In [7]:
state = 'empty'
total_profit = 0
cnt = 0
takercnt = 0

data['mid_fu'] = data['ap1_fu']/2+data['bp1_fu']/2
data['predict1']=pd.DataFrame(y, index = data.index)
data['stdp2']=pd.DataFrame(y3, index = data.index)
data['stdm2']=pd.DataFrame(y4, index = data.index)

p1='predict1'
p2='predict1'
stp='stdp2'
stm='stdm2'

resl = []
def select(x):
    return (1.0+np.maximum(x,0)/2)*1
def select2(x):
    return (1.0+np.maximum(x,0)/2)*1
buy_price_lock = 0
sell_price_lock = 0
lag = 0 # time lag cuz in real trading there will be network lag
filename='fu'
for i in range(0, len(data)-1-lag):
    current_price = data['mid_'+filename][i]
    if state == 'empty':
        buy_price = np.minimum(data['bp1_'+filename][i]*(1-np.maximum(select(data[stm][i]),0)/1e4)*(1+np.clip(4*(data[p1][i]-0.0)/1e4, -20/1e4, 0/1e4)), data['bp1_'+filename][i])
        sell_price = np.maximum(data['ap1_'+filename][i]*(1+np.maximum(select(data[stp][i]),0)/1e4)*(1+np.clip(4*(data[p1][i]+0.0)/1e4,-0/1e4, 20/1e4)), data['ap1_'+filename][i])
        tpb = data['sellpriceT_'+filename][i+1]
        tps = data['buypriceT_'+filename][i+1]
        #print(data['time'][i], buy_price, tp)
        if tpb <= buy_price-0.01 and tpb != 0 and data[p1][i]>-1.0:
            #print(data['predict3'][i])
            state = 'buy'
            buy_price_lock = buy_price
            print(data.index[i],data[p1][i])
            print('buy',i, buy_price, tpb, data['buyprice_'+filename][i+1], data['mid_'+filename][i])
            total_profit += 0.5/1e4
            continue
        if tps >= sell_price+0.01 and tps != 0 and data[p1][i]<1.0:
            #print(data['predict3'][i])
            state = 'sell'
            sell_price_lock = sell_price
            print(data.index[i],data[p1][i])
            print('sell',i, sell_price, data['mid_'+filename][i])
            total_profit += 0.5/1e4
            continue
    
    if state == 'buy' and (data[p1][i]+data[p2][i])/2 < -10:
        sell_price = data['bp1_fu'][i+5]*(1.0)#(current_price)
        state = 'empty'
        print(data.index[i],data[p1][i])
        print('sell close', sell_price)
        total_profit += sell_price / buy_price_lock - 1 - 2/1e4
        cnt += 1
        takercnt += 1
        resl.append((data.index[i], total_profit))
        continue
    
    if state == 'sell' and (data[p1][i]+data[p2][i])/2 > 10:
        buy_price = data['ap1_fu'][i+5]*(1.0)#(current_price)
        state = 'empty'
        print(data.index[i],data[p1][i])
        print('buy close', buy_price)
        total_profit += sell_price_lock / buy_price - 1 - 2/1e4
        cnt += 1
        takercnt += 1
        resl.append((data.index[i], total_profit))
        continue
    
    if state == 'buy':
        sell_price = np.maximum(data['ap1_'+filename][i]*(1+np.maximum(select2(data[stp][i]),0)/2e4)*(1+np.clip(3*(data[p1][i]+0.0)/1e4,-0/1e4, 20/1e4)), data['ap1_'+filename][i])
        tp = data['buypriceT_'+filename][i+1]
        if tp >= sell_price+0.01 and tp != 0:
            state = 'empty'
            print(data.index[i],data[p1][i])
            print('sell close', sell_price)
            total_profit += sell_price / buy_price_lock - 1 + 0.5/1e4
            cnt += 1
            resl.append((data.index[i], total_profit))
            continue
    
    if state == 'sell':
        buy_price = np.minimum(data['bp1_'+filename][i]*(1-np.maximum(select2(data[stm][i]),0)/2e4)*(1+np.clip(3*(data[p1][i]-0.0)/1e4, -20/1e4, 0/1e4)), data['bp1_'+filename][i])
        tp = data['sellpriceT_'+filename][i+1]
        if tp <= buy_price-0.01 and tp != 0:
            state = 'empty'
            print(data.index[i],data[p1][i])
            print('buy close', buy_price, tp)
            total_profit += sell_price_lock / buy_price - 1 + 0.5/1e4
            cnt += 1
            resl.append((data.index[i], total_profit))
        continue
print(total_profit, cnt, takercnt)

2022-07-22 08:11:21.944223 1.1328153163780996
buy 652 1605.8145393641998 1605.800048828125 0.0 1606.155
2022-07-22 08:11:38.992301 -1.624940936659769
sell close 1610.1730276870114
2022-07-22 08:11:46.959789 -1.4130336069229088
sell 900 1610.7257423204337 1610.125
2022-07-22 08:11:48.499461 1.9286048557454194
buy close 1609.339450040321 1609.300048828125
2022-07-22 08:12:04.039765 3.3824753625033566
buy 1069 1608.0741984217232 1608.02001953125 1608.4599609375 1608.455
2022-07-22 08:12:15.202449 0.8070631620871183
sell close 1612.815914460093
2022-07-22 08:12:25.703202 -1.0074691552830473
sell 1283 1613.5410675577182 1613.155
2022-07-22 08:12:26.007890 0.014500321006138961
buy close 1612.789364661017 1612.5799560546875
2022-07-22 08:12:30.046755 -0.7165658681795861
sell 1326 1613.1864275633577 1612.805
2022-07-22 08:12:41.028834 2.4767168985118104
buy close 1610.332014093017 1610.260009765625
2022-07-22 08:13:24.821279 0.5238805000118418
buy 1868 1610.1737729959266 1610.1500244140625 0.0

In [8]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(*zip(*resl), color='red')
ax2.plot(data.index, (data['mid_fu']), color='blue')
plt.show()